In [1]:
import os
!pip3 install glob
import glob
import dill
import pathlib
import pandas as pd
import numpy as np
!pip3 install pydub
import pydub
!pip3 install python_speech_features
import python_speech_features
from utils import * 
import re

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#calculating mfccs

In [0]:
base_folder = pathlib.Path("/content/drive/My Drive/validated_clips/validated_clip/")
out_folder =  pathlib.Path("/content/drive/My Drive/validated_clips/preprocessed/")
outfiles = os.listdir(out_folder)
already_done = set()

for file in outfiles:
  thing = re.compile('[a-z]')
  already_done.add(file[:thing.search(file).start()])

folders = os.listdir(base_folder)

In [29]:
todo = ["2.000000"]
for folder in folders:
  #if folder in already_done:
  #  print(f'skipping {folder}')
  # continue
  if folder not in todo:
    print(f'skipping{folder}')
    continue

  print(folder)
  inner_folder = base_folder / folder
  globby = str(inner_folder)+"/*.mp3"
  downloaded_files = glob.glob(globby)
  #print(downloaded_files)

  filepath_dict = {pathlib.Path(filepath).name : filepath for filepath in downloaded_files}
  print(filepath_dict)

  dev = pd.read_csv("/content/drive/My Drive/tsvs/dev.tsv", sep="\t")
  train = pd.read_csv("/content/drive/My Drive/tsvs/train.tsv", sep="\t")
  test = pd.read_csv("/content/drive/My Drive/tsvs/test.tsv", sep="\t")
  splits = {
      "dev": dev,
      "train": train, 
      "test": test
  }

  #only files in current folder
  for key in splits:
    print(key)
    print(splits[key].shape)
    splits[key]["path"] = splits[key]["path"].map(filepath_dict)
    splits[key] = splits[key][splits[key]['path'].notna()]
    print(splits[key].shape)

  #only ones with accents
  for key in splits:
    splits[key].dropna(axis=0, subset=["accent"], inplace=True)
  

  downloaded_audios_dict = {key : [pydub.AudioSegment.from_mp3(f) for f in splits[key]['path']] for key in splits}
  max_lengths = [np.max([len(sample) for sample in samples]) for samples in downloaded_audios_dict.values() if samples != []]
  if max_lengths == []:
    print("I cry")
    continue
  max_audio_length = np.max(max_lengths)
  padded_audio_dict = {key: [zero_pad_in_end(audio, max_audio_length) for audio in audios] for key, audios in downloaded_audios_dict.items()}

  labels_dict = {key: splits[key]['accent'].to_numpy() for key in splits}
  paths_dict ={key: [f for f in splits[key]['path']] for key in splits}
  mfccs_padded_dict = {key: np.array([extract_mfcc(audio) for audio in padded_audios]) for key, padded_audios in padded_audio_dict.items()}

  label_filename = folder+"label.pkl"
  paths_filename = folder+"paths.pkl"
  mfccs_padded_filename = folder+"mfccs_padded.pkl"

  label_filepath = out_folder / label_filename
  paths_filepath = out_folder / paths_filename
  mfccs_padded_filepath = out_folder / mfccs_padded_filename

  with open(label_filepath , "wb") as f:
    dill.dump(labels_dict, f)
  with open(paths_filepath , "wb") as f:
    dill.dump(paths_dict, f)
  with open(mfccs_padded_filepath , "wb") as f:
    dill.dump(mfccs_padded_dict, f)

skipping103.000000
skipping101.000000
skipping100.000000
skipping105.000000
skipping104.000000
skipping106.000000
skipping1.000000
skipping107.000000
skipping102.000000
skipping10.000000
skipping114.000000
skipping116.000000
skipping11.000000
skipping109.000000
skipping115.000000
skipping110.000000
skipping111.000000
skipping112.000000
skipping113.000000
skipping108.000000
skipping120.000000
skipping122.000000
skipping117.000000
skipping124.000000
skipping119.000000
skipping125.000000
skipping121.000000
skipping12.000000
skipping123.000000
skipping118.000000
skipping127.000000
skipping130.000000
skipping129.000000
skipping134.000000
skipping128.000000
skipping13.000000
skipping133.000000
skipping131.000000
skipping126.000000
skipping132.000000
skipping142.000000
skipping138.000000
skipping141.000000
skipping14.000000
skipping135.000000
skipping143.000000
skipping140.000000
skipping137.000000
skipping136.000000
skipping139.000000
skipping145.000000
skipping151.000000
skipping148.000000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


skipping206.000000
skipping200.000000
skipping202.000000
skipping199.000000
skipping20.000000
skipping205.000000
skipping22.000000
skipping27.000000
skipping26.000000
skipping21.000000
skipping25.000000
skipping207.000000
skipping24.000000
skipping23.000000
skipping208.000000
skipping28.000000
skipping35.000000
skipping34.000000
skipping37.000000
skipping33.000000
skipping36.000000
skipping3.000000
skipping29.000000
skipping32.000000
skipping30.000000
skipping31.000000
skipping44.000000
skipping43.000000
skipping39.000000
skipping38.000000
skipping46.000000
skipping4.000000
skipping42.000000
skipping40.000000
skipping45.000000
skipping41.000000
skipping5.000000
skipping51.000000
skipping52.000000
skipping48.000000
skipping47.000000
skipping55.000000
skipping50.000000
skipping53.000000
skipping49.000000
skipping54.000000
skipping60.000000
skipping57.000000
skipping61.000000
skipping6.000000
skipping58.000000
skipping64.000000
skipping62.000000
skipping63.000000
skipping59.000000
skippin

In [16]:
#SAVE GENDER

wanted_accents=["england", "indian", "us", "scotland", "australia"]
preprocessed_folder = pathlib.Path('/content/drive/My Drive/validated_clips/preprocessed/')

for folder in folders:
  print(folder)
  inner_folder = base_folder / folder
  globby = str(inner_folder)+"/*.mp3"
  downloaded_files = glob.glob(globby)
  #print(downloaded_files)

  filepath_dict = {pathlib.Path(filepath).name : filepath for filepath in downloaded_files}
  #print(filepath_dict)

  dev = pd.read_csv("/content/drive/My Drive/tsvs/dev.tsv", sep="\t")
  train = pd.read_csv("/content/drive/My Drive/tsvs/train.tsv", sep="\t")
  test = pd.read_csv("/content/drive/My Drive/tsvs/test.tsv", sep="\t")
  splits = {
      "dev": dev,
      "train": train, 
      "test": test
  }

  #only files in current folder
  for key in splits:
    splits[key]["path"] = splits[key]["path"].map(filepath_dict)
    splits[key] = splits[key][splits[key]['path'].notna()]

  #only ones with accents
  for key in splits:
    splits[key].dropna(axis=0, subset=["accent"], inplace=True)
  
  #only ones in wanted accents
  for key in splits:
    splits[key].loc[splits[key]['accent'].isin(wanted_accents)]


  genders_dict = {key: splits[key]['gender'].to_numpy() for key in splits}
  genders_filename = folder+"genders_5accents.pkl"
  genders_filepath = preprocessed_folder / genders_filename

  with open(genders_filepath , "wb") as f:
    dill.dump(genders_dict, f)

103.000000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


101.000000
100.000000
105.000000
104.000000
106.000000
1.000000
107.000000
102.000000
10.000000
114.000000
116.000000
11.000000
109.000000
115.000000
110.000000
111.000000
112.000000
113.000000
108.000000
120.000000
122.000000
117.000000
124.000000
119.000000
125.000000
121.000000
12.000000
123.000000
118.000000
127.000000
130.000000
129.000000
134.000000
128.000000
13.000000
133.000000
131.000000
126.000000
132.000000
142.000000
138.000000
141.000000
14.000000
135.000000
143.000000
140.000000
137.000000
136.000000
139.000000
145.000000
151.000000
148.000000
149.000000
144.000000
152.000000
146.000000
147.000000
150.000000
15.000000
159.000000
160.000000
158.000000
16.000000
153.000000
156.000000
155.000000
161.000000
154.000000
157.000000
169.000000
168.000000
165.000000
166.000000
162.000000
167.000000
163.000000
164.000000
17.000000
170.000000
177.000000
172.000000
178.000000
18.000000
174.000000
176.000000
175.000000
179.000000
171.000000
173.000000
188.000000
187.000000
181.000000

In [17]:
#SAVE speaker

wanted_accents=["england", "indian", "us", "scotland", "australia"]
preprocessed_folder = pathlib.Path('/content/drive/My Drive/validated_clips/preprocessed/')

for folder in folders:
  print(folder)
  inner_folder = base_folder / folder
  globby = str(inner_folder)+"/*.mp3"
  downloaded_files = glob.glob(globby)
  #print(downloaded_files)

  filepath_dict = {pathlib.Path(filepath).name : filepath for filepath in downloaded_files}
  #print(filepath_dict)

  dev = pd.read_csv("/content/drive/My Drive/tsvs/dev.tsv", sep="\t")
  train = pd.read_csv("/content/drive/My Drive/tsvs/train.tsv", sep="\t")
  test = pd.read_csv("/content/drive/My Drive/tsvs/test.tsv", sep="\t")
  splits = {
      "dev": dev,
      "train": train, 
      "test": test
  }

  #only files in current folder
  for key in splits:
    splits[key]["path"] = splits[key]["path"].map(filepath_dict)
    splits[key] = splits[key][splits[key]['path'].notna()]

  #only ones with accents
  for key in splits:
    splits[key].dropna(axis=0, subset=["accent"], inplace=True)
  
  #only ones in wanted accents
  for key in splits:
    splits[key].loc[splits[key]['accent'].isin(wanted_accents)]


  speakers_dict = {key: splits[key]['client_id'].to_numpy() for key in splits}
  speakers_filename = folder+"speakers_5accents.pkl"
  speakers_filepath = preprocessed_folder / speakers_filename
  print(speakers_filepath)

  with open(speakers_filepath , "wb") as f:
    dill.dump(speakers_dict, f)

103.000000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


/content/drive/My Drive/validated_clips/preprocessed/103.000000speakers_5accents.pkl
101.000000
/content/drive/My Drive/validated_clips/preprocessed/101.000000speakers_5accents.pkl
100.000000
/content/drive/My Drive/validated_clips/preprocessed/100.000000speakers_5accents.pkl
105.000000
/content/drive/My Drive/validated_clips/preprocessed/105.000000speakers_5accents.pkl
104.000000
/content/drive/My Drive/validated_clips/preprocessed/104.000000speakers_5accents.pkl
106.000000
/content/drive/My Drive/validated_clips/preprocessed/106.000000speakers_5accents.pkl
1.000000
/content/drive/My Drive/validated_clips/preprocessed/1.000000speakers_5accents.pkl
107.000000
/content/drive/My Drive/validated_clips/preprocessed/107.000000speakers_5accents.pkl
102.000000
/content/drive/My Drive/validated_clips/preprocessed/102.000000speakers_5accents.pkl
10.000000
/content/drive/My Drive/validated_clips/preprocessed/10.000000speakers_5accents.pkl
114.000000
/content/drive/My Drive/validated_clips/prepro

In [0]:
print(len(already_done))

208


In [0]:
print(len(folders))

208


#fixing preprocessing

In [0]:
preprocessed_folder = pathlib.Path('/content/drive/My Drive/validated_clips/preprocessed/')


In [8]:
mfccs_padded = {"dev":[], "train":[], "test":[]}
for i in range(1,209):
  number = '{:.6f}'.format(i)
  mfcc_filename = number+"mfccs_padded.pkl"
  mfcc_filepath = preprocessed_folder / mfcc_filename

  with open(mfcc_filepath, "rb") as f:
    mfcc_local = dill.load(f)
  mfccs_padded['dev'].extend(mfcc_local['dev'])
  mfccs_padded['train'].extend(mfcc_local['train'])
  mfccs_padded['test'].extend(mfcc_local['test'])

KeyboardInterrupt: ignored

In [0]:
max_lengths = [np.max([len(sample) for sample in samples]) for samples in mfccs_padded.values() if samples != []]
print(max_lengths)
max_length = np.max(max_lengths)

In [5]:
print(max_length)

6234


In [10]:
wanted_accents=["england", "indian", "us", "scotland", "australia"]

for folder_num in range(1,209):
  number = '{:.6f}'.format(folder_num)
  mfcc_filename = number+"mfccs_padded_real.pkl"
  label_filename = number+"label.pkl"
  paths_filename = number+"paths.pkl"
  mfcc_filepath = preprocessed_folder / mfcc_filename
  label_filepath = preprocessed_folder / label_filename
  paths_filepath = preprocessed_folder / paths_filename

  with open(mfcc_filepath, "rb") as f:
    mfcc_local = dill.load(f)
  with open(label_filepath, "rb") as f:
    label_local = dill.load(f)
  with open(paths_filepath , "rb") as f:
    path_local = dill.load(f)

  new_labels = {key: [label_local[key][i] for i in range(len(label_local[key])) if label_local[key][i] in wanted_accents] for key in label_local}
  new_mfccs = {key: [mfcc_local[key][i] for i in range(len(label_local[key])) if label_local[key][i] in wanted_accents] for key in label_local}
  new_paths = {key: [path_local[key][i] for i in range(len(label_local[key])) if label_local[key][i] in wanted_accents] for key in label_local}

  

  new_label_filename = number+"label_5accents.pkl"
  new_mfcc_filename = number+"mfcc_5accents.pkl"
  new_path_filename = number+"path_5accents.pkl"

  new_label_filepath = preprocessed_folder / new_label_filename
  new_mfccs_filepath = preprocessed_folder / new_mfcc_filename
  new_paths_filepath = preprocessed_folder / new_path_filename

  print(folder_num, [len(value) for value in new_mfccs.values()],'\n', new_labels)

  with open(new_label_filepath , "wb") as f:
    dill.dump(new_labels, f)
  with open(new_paths_filepath , "wb") as f:
    dill.dump(new_paths, f)
  with open(new_mfccs_filepath , "wb") as f:
    dill.dump(new_mfccs, f)

1 [2, 12, 1] 
 {'dev': ['us', 'us'], 'train': ['us', 'indian', 'indian', 'indian', 'indian', 'indian', 'indian', 'indian', 'indian', 'indian', 'us', 'us'], 'test': ['us']}
2 [1, 4, 2] 
 {'dev': ['us'], 'train': ['australia', 'us', 'australia', 'scotland'], 'test': ['us', 'us']}
3 [0, 1, 4] 
 {'dev': [], 'train': ['us'], 'test': ['us', 'indian', 'us', 'us']}
4 [2, 3, 1] 
 {'dev': ['us', 'us'], 'train': ['england', 'england', 'us'], 'test': ['scotland']}
5 [0, 18, 3] 
 {'dev': [], 'train': ['indian', 'indian', 'indian', 'indian', 'indian', 'us', 'us', 'us', 'us', 'us', 'us', 'us', 'us', 'us', 'us', 'us', 'england', 'england'], 'test': ['england', 'england', 'england']}
6 [1, 2, 3] 
 {'dev': ['us'], 'train': ['indian', 'indian'], 'test': ['us', 'us', 'us']}
7 [1, 7, 0] 
 {'dev': ['us'], 'train': ['us', 'us', 'us', 'us', 'us', 'us', 'indian'], 'test': []}
8 [0, 32, 5] 
 {'dev': [], 'train': ['us', 'us', 'us', 'us', 'us', 'us', 'us', 'us', 'us', 'us', 'us', 'us', 'us', 'us', 'us', 'us', 'us

In [0]:
overall_padded= {key: np.stack( overall_padded[key], axis=0 ) for key in overall_padded.keys()}

In [1]:
print(stackyboi)

NameError: ignored

In [0]:

overall_filepath = preprocessed_folder / "overall_mfcc_padded.pkl"
with open(mfcc_filepath, "wb") as f:
    mfcc_local = dill.write(overall_padded, f)